<td>
   <a target="_blank" href="https://labelbox.com" ><img src="https://labelbox.com/blog/content/images/2021/02/logo-v4.svg" width=256/></a>
</td>

<td>
<a href="https://colab.research.google.com/github/Labelbox/labelbox-python/blob/master/examples/prediction_upload/video_predictions.ipynb" target="_blank"><img
src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"></a>
</td>

<td>
<a href="https://github.com/Labelbox/labelbox-python/blob/master/examples/prediction_upload/video_predictions.ipynb" target="_blank"><img
src="https://img.shields.io/badge/GitHub-100000?logo=github&logoColor=white" alt="GitHub"></a>
</td>

# Video Prediction Import 
* This notebook walks you through the process of uploading model predictions to a Model Run. This notebook provides an example for each supported prediction type for video assets.

A Model Run is a container for the predictions, annotations and metrics of a specific experiment in your ML model development cycle.

**Supported annotations that can be uploaded through the SDK**
- Bounding box
- Point
- Polyline
- Classification - radio
- Classification - checklist
- Classification - free text
- Nested classifications 

**NOT** supported:
- Polygons [not supported in video editor or model]
- Raster segmentation masks [not supported in model]
- Vector segmentation masks [not supported in video editor]



## Setup

In [ ]:
!pip install -q "labelbox[data]"

In [ ]:
import labelbox as lb
import labelbox.types as lb_types
import uuid

## Replace with your API Key 
Guides on [Create an API key](https://docs.labelbox.com/docs/create-an-api-key)

In [ ]:
API_KEY=""
client = lb.Client(API_KEY)

## Supported Predictions
- Confidence scores are currently not supported for segment or frame annotations, which are required for bounding box, point, and line for video assets. For this tutorial, only the radio and checklist annotations will have confidence scores.

In [ ]:
####### Bounding box (frame specific) ###########

# Confidence scores are not supported for frame specific bounding box annotations and VideoObjectAnnotation

# bbox dimensions 
bbox_dm = {
  "top":617,
  "left":1371,
  "height":419,
  "width":505
}

# Python Annotation
bbox_prediction = [
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=13,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]), # x= left + width , y = top + height
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=15,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]),
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]),
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=19,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]), 
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]),
      )
  )
]


# NDJSON
bbox_prediction_ndjson = {
    "name" : "bbox_video",
    "segments" : [{
        "keyframes" : [
            {
              "frame": 13,
              "bbox" : bbox_dm 
           },
           {
              "frame": 15,
              "bbox" : bbox_dm 
           },
           {
              "frame": 19,
              "bbox" : bbox_dm 
           }
        ]
      }
    ]
}

In [ ]:
######## Point ########
# Confidence score is not supported for VideoObjectAnnotation
# Python Annotation
point_prediction = [
    lb_types.VideoObjectAnnotation(
        name = "point_video",
        keyframe=True,
        frame=17,
        value = lb_types.Point(x=660.134, y=407.926),
        )
]

# NDJSON
point_prediction_ndjson = {
    "name": "point_video", 
    "confidence": 0.5,
    "segments": [{
        "keyframes": [{
            "frame": 17,
            "point" : {
                "x": 660.134 ,
                "y": 407.926
            }
        }]
    }] 
}

In [ ]:
######## Polyline (frame specific) ########
# confidence scores are not supported in polyline annotations

# Python Annotation
polyline_prediction = [
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=5,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=12,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=20,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=24,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=45,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  )
  
]

# NDJSON
polyline_prediction_ndjson = {
  "name": "line_video_frame", 
  "segments": [
      {
        "keyframes": [
          {
            "frame": 5,
            "line": [{
              "x": 680,
              "y": 100
            },{
              "x": 100,
              "y": 190
            },{
              "x": 190,
              "y": 220
            }]
          },
          {
            "frame": 12,
            "line": [{
              "x": 680,
              "y": 280
            },{
              "x": 300,
              "y": 380
            },{
              "x": 400,
              "y": 460
            }]
          },
          {
            "frame": 20,
            "line": [{
              "x": 680,
              "y": 180
            },{
              "x": 100,
              "y": 200
            },{
              "x": 200,
              "y": 260
            }]
          }
        ]
      },
      {
        "keyframes": [
          {
            "frame": 24,
            "line": [{
              "x": 300,
              "y": 310
            },{
              "x": 330,
              "y": 430
            }]
          },
          {
            "frame": 45,
            "line": [{
              "x": 600,
              "y": 810
            },{
              "x": 900,
              "y": 930
            }]
          }
        ]
      }
    ]
}

In [ ]:
######## Frame base classifications ########

# Python Annotation
radio_prediction = [
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=9,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(
          name = "first_radio_answer",
          confidence=0.5 
          ))
    ),
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=15,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(
          name = "first_radio_answer",
          confidence=0.5 
          ))
    )
]

checklist_prediction= [
    lb_types.VideoClassificationAnnotation(
        name="checklist_class",
        frame=29,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer",
                    confidence=0.5 
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=35,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer",
                    confidence=0.5 
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=39, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer",
                    confidence=0.5 
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=45, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer",
                    confidence=0.5 
                )
            ]
            )
        )
]


## NDJSON
frame_radio_classification_prediction_ndjson = {
    "name": "radio_class", 
    "answer": { "name": "first_radio_answer", "frames": [{"start": 9, "end": 15}]}
}

## frame specific
frame_checklist_classification_prediction_ndjson = {
    "name": "checklist_class", 
    "answer": [
        { "name": "first_checklist_answer" , "frames": [{"start": 29, "end": 35 }]},
        { "name": "second_checklist_answer", "frames": [{"start": 39, "end": 45 }]} 
  ]      
}

In [ ]:
####### Global Classifications ######### 

# Python Annotation
## For global classifications use ClassificationAnnotation
global_radio_prediction = [lb_types.ClassificationAnnotation(
    name="radio_class_global",
    value=lb_types.Radio(answer = lb_types.ClassificationAnswer(
      name = "first_radio_answer",
      confidence=0.5 
      ))
)]

global_checklist_prediction=[lb_types.ClassificationAnnotation(
  name="checklist_class_global", 
  value=lb_types.Checklist(
      answer = [
        lb_types.ClassificationAnswer(
            name = "first_checklist_answer",
            confidence=0.5 
        ), 
        lb_types.ClassificationAnswer(
            name = "second_checklist_answer",
            confidence=0.5 
        )
      ]
    )
 )]

# NDJSON
global_radio_classification_ndjson = {
    "name": "radio_class_global", 
    "answer": { "name": "first_radio_answer", "confidence": 0.5}
}


global_checklist_classification_ndjson = {
    "name": "checklist_class_global", 
    "answer": [
        { "name": "first_checklist_answer" , "confidence": 0.5},
        { "name": "second_checklist_answer", "confidence": 0.5} 
  ]      
}

In [ ]:
########## Nested Global Classification ########### 

# Python Annotation
nested_radio_prediction =[lb_types.ClassificationAnnotation(
  name="nested_radio_question",
  value=lb_types.Radio(
    answer=lb_types.ClassificationAnswer(
      name="first_radio_answer",
      confidence=0.5 ,
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_radio_question",
          value=lb_types.Radio(
            answer=lb_types.ClassificationAnswer(
              name="first_sub_radio_answer",
              confidence=0.5 
            )
          )
        )
      ]
    )
  )
)]

# NDJSON
nested_radio_prediction_ndjson = {
  "name": "nested_radio_question",
  "answer": {"name": "first_radio_answer", "confidence": 0.5,
  "classifications" : [
    {"name": "sub_radio_question", "answer": {"name": "first_sub_radio_answer", "confidence": 0.5}}
   ]
  }
}

# Python Annotation
nested_checklist_prediction = [lb_types.ClassificationAnnotation(
  name="nested_checklist_question",
  value=lb_types.Checklist(
    answer=[lb_types.ClassificationAnswer(
      name="first_checklist_answer",
      confidence=0.5 ,
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_checklist_question",
          value=lb_types.Checklist(
            answer=[lb_types.ClassificationAnswer(
            name="first_sub_checklist_answer",
            confidence=0.5 
          )]
        ))
      ]
    )]
  )
)]

# NDJSON
nested_checklist_prediction_ndjson = {
  "name": "nested_checklist_question",
  "answer": [{
      "name": "first_checklist_answer",
      "confidence": 0.5,
      "classifications" : [
        {
          "name": "sub_checklist_question",
          "answer": {"name": "first_sub_checklist_answer", "confidence": 0.5}
        }
      ]
  }]
}

In [ ]:
########## Classifications under frame base tools ##########
# Confidence scores are not supported for frame specific bounding box annotations with sub-classifications

# bounding box dimensions
bbox_dm2 = {
  "top": 146.0,
  "left": 98.0,
  "height": 382.0,
  "width": 341.0
}

# Python Annotation 
frame_bbox_with_checklist_subclass_prediction = [
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=10,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]), # x= left + width , y = top + height
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=11,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]),
      ),
    classifications=[
                lb_types.ClassificationAnnotation(
                    name='checklist_class',
                    value=lb_types.Checklist(answer=[lb_types.ClassificationAnswer(
                        name="first_checklist_answer",
                        confidence=0.5 
                    )])
                )
            ]
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=13,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]),
      ),
    classifications=[
              lb_types.ClassificationAnnotation(
                  name='checklist_class',
                  value=lb_types.Checklist(answer=[lb_types.ClassificationAnswer(
                      name="second_checklist_answer",
                      confidence=0.5 
                  )])
              )
          ]
  )
]

frame_bbox_with_checklist_subclass_prediction_ndjson = {
    "name": "bbox_class",
    "segments": [{
        "keyframes": [
            {
            "frame": 10,
            "bbox": bbox_dm2
          },
          {  
          "frame": 11,
            "bbox": bbox_dm2,
            "classifications": [
              {
                "name": "bbox_radio",
                "answer": [{"name": "first_checklist_answer", "confidence": 0.5}]
              }
            ]
          },
          {  
          "frame": 13,
            "bbox": bbox_dm2,
            "classifications": [
              {
                "name": "bbox_radio",
                "answer": [{"name": "second_checklist_answer", "confidence": 0.5}]
              }
            ]
          }
        ]
      }
    ]
}

In [ ]:
######### Free text classification ###########
text_prediction = [lb_types.ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=lb_types.Text(answer="sample text", confidence=0.5)
)]

text_prediction_ndjson = {
  'name': 'free_text',
  'confidence': 0.5,
  'answer': 'sample text',
}

## Step 1: Import data rows into Catalog

In [ ]:
# send a sample image as batch to the project
global_key = "sample-video-2.mp4"
test_img_url = {
    "row_data": "https://storage.googleapis.com/labelbox-datasets/video-sample-data/sample-video-2.mp4",
    "global_key": global_key
}
dataset = client.create_dataset(
    name="Video prediction demo",
    iam_integration=None # Removing this argument will default to the organziation's default iam integration
)
task = dataset.create_data_rows([test_img_url])
task.wait_till_done()
print("Errors: ",task.errors)
print("Failed data rows: ",task.failed_data_rows)

## Step 2: Create/select an Ontology for your model predictions
Your project should have the correct ontology setup with all the tools and classifications supported for your annotations, and the tool names and classification instructions should match the name/instructions fields in your annotations to ensure the correct feature schemas are matched.


In [ ]:
ontology_builder = lb.OntologyBuilder(
    tools=[
        lb.Tool(tool=lb.Tool.Type.BBOX, name="bbox_video"),
        lb.Tool(tool=lb.Tool.Type.POINT, name="point_video"),
        lb.Tool(tool=lb.Tool.Type.LINE, name="line_video_frame"),
        lb.Tool(tool=lb.Tool.Type.RASTER_SEGMENTATION, name="video_mask"),
        lb.Tool(
          tool=lb.Tool.Type.BBOX, name="bbox_class",
          classifications=[
            lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist_class",
            scope = lb.Classification.Scope.INDEX, ## defined scope for frame classifications
            options=[ 
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
            ]
            )
          ]
        )
    ],
    classifications=[ 
        lb.Classification(
            class_type=lb.Classification.Type.CHECKLIST,
            name="checklist_class",
            scope = lb.Classification.Scope.INDEX, ## defined scope for frame classifications
            options=[ 
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
            ]
        ),
        lb.Classification(
            class_type=lb.Classification.Type.RADIO,
            name="radio_class",
            scope = lb.Classification.Scope.INDEX,
            options=[ 
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer")
            ]
        ),
         lb.Classification(
              class_type=lb.Classification.Type.RADIO,
              name="nested_radio_question",
              options=[
                  lb.Option("first_radio_answer",
                        options=[
                            lb.Classification(
                                class_type=lb.Classification.Type.RADIO,
                                name="sub_radio_question",
                                options=[lb.Option("first_sub_radio_answer")]
                            )
                        ]
                  )
              ] 
        ),
        lb.Classification(
          class_type=lb.Classification.Type.CHECKLIST,
          name="nested_checklist_question",
          options=[
              lb.Option("first_checklist_answer",
                options=[
                  lb.Classification(
                      class_type=lb.Classification.Type.CHECKLIST,
                      name="sub_checklist_question",
                      options=[lb.Option("first_sub_checklist_answer")]
                  )
              ]
            )
          ]
        ),
        lb.Classification(
          class_type=lb.Classification.Type.RADIO, 
          name="radio_class_global",
          options=[ 
                lb.Option(value="first_radio_answer"),
                lb.Option(value="second_radio_answer")
            ]
        ),
        lb.Classification(
          class_type=lb.Classification.Type.CHECKLIST,
          name="checklist_class_global",
          options=[
                lb.Option(value="first_checklist_answer"),
                lb.Option(value="second_checklist_answer")
          ]
        ),
        lb.Classification(
          class_type=lb.Classification.Type.TEXT,
          name="free_text"
        )
    ]  
)

ontology = client.create_ontology("Ontology Video Annotations", 
                                  ontology_builder.asdict(), 
                                  media_type=lb.MediaType.Video
                                  )

## Step 3: Create a Model and Model Run

In [ ]:
# create Model
model = client.create_model(name="video_model_run_" + str(uuid.uuid4()), 
                            ontology_id=ontology.uid)
# create Model Run
model_run = model.create_model_run("iteration 1")

## Step 4: Send data rows to the Model Run

In [ ]:
model_run.upsert_data_rows(global_keys=[global_key])

## Step 5. Create the predictions payload

Create the annotations payload using the snippets of [code here](https://docs.labelbox.com/reference/import-video-annotations).



#### Python Annotation Types

In [ ]:
label_predictions = []
annotations_list = [
    point_prediction,
    bbox_prediction,
    polyline_prediction, 
    checklist_prediction, 
    radio_prediction,
    nested_radio_prediction,
    nested_checklist_prediction,
    frame_bbox_with_checklist_subclass_prediction,
    global_radio_prediction,
    global_checklist_prediction,
    text_prediction
      ]

flatten_list_annotations = [ann for ann_sublist in annotations_list for ann in ann_sublist] 

label_predictions.append(
    lb_types.Label(
        data=lb_types.VideoData(global_key=global_key),
        annotations = flatten_list_annotations
    )
)

#### NDJSON annotations

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations
label_prediction_ndjson = []

for annotation in [
    point_prediction_ndjson,
    bbox_prediction_ndjson,
    polyline_prediction_ndjson, 
    frame_checklist_classification_prediction_ndjson, 
    frame_radio_classification_prediction_ndjson,
    nested_radio_prediction_ndjson,
    nested_checklist_prediction_ndjson,
    frame_bbox_with_checklist_subclass_prediction_ndjson,
    global_radio_classification_ndjson,
    global_checklist_classification_ndjson,
    text_prediction_ndjson
]:      
  annotation.update({
      "dataRow": {
          "globalKey": global_key
      }
  })
  label_prediction_ndjson.append(annotation)

## Step 6. Upload the predictions payload to the Model Run 

In [ ]:
# Upload the prediction label to the Model Run
upload_job_prediction = model_run.add_predictions(
    name="prediction_upload_job"+str(uuid.uuid4()),
    predictions=label_predictions)

# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_prediction.errors)
print("Status of uploads: ", upload_job_prediction.statuses)

## Step 7: Send annotations to the Model Run 
To send annotations to a Model Run, we must first import them into a project, create a label payload and then send them to the Model Run.

##### 7.1. Create a labelbox project

In [ ]:
# Create a Labelbox project
project = client.create_project(name="video_prediction_demo",
                                    auto_audit_percentage=1,
                                    auto_audit_number_of_labels=1,
                                    media_type=lb.MediaType.Video)
project.setup_editor(ontology)

##### 7.2. Create a batch to send to the project 

In [ ]:
project.create_batch(
  "batch_video_prediction_demo", # Each batch in a project must have a unique name
  global_keys=[global_key], # A list of data rows, data row ids or global keys
  priority=5 # priority between 1(Highest) - 5(lowest)
)

##### 7.3 Create the annotations payload

In [ ]:
# Python Annotation
point_annotation = [
    lb_types.VideoObjectAnnotation(
        name = "point_video",
        keyframe=True,
        frame=17,
        value = lb_types.Point(x=660.134, y=407.926),
        )
]


######## Polyline ########

# Python Annotation
polyline_annotation = [
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=5,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=12,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=20,
    segment_index=0,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=24,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "line_video_frame",
    keyframe=True, 
    frame=45,
    segment_index=1,
    value=lb_types.Line( 
          points=[lb_types.Point(x=680, y=100), lb_types.Point(x=100, y=190)]
      )
  )
  
]

radio_annotation = [
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=9,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
    ),
    lb_types.VideoClassificationAnnotation(
        name="radio_class", 
        frame=15,
        segment_index=0,
        value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
    )
]

checklist_annotation= [
    lb_types.VideoClassificationAnnotation(
        name="checklist_class",
        frame=29,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=35,
        segment_index=0,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "first_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=39, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer"
                )
            ]
            )
        ),
    lb_types.VideoClassificationAnnotation(
        name="checklist_class", 
        frame=45, 
        segment_index=1,
        value=lb_types.Checklist(
            answer = [
                
                lb_types.ClassificationAnswer(
                    name = "second_checklist_answer"
                )
            ]
            )
        )
]

global_radio_annotation = [lb_types.ClassificationAnnotation(
    name="radio_class_global",
    value=lb_types.Radio(answer = lb_types.ClassificationAnswer(name = "first_radio_answer"))
)]

global_checklist_annotation=[lb_types.ClassificationAnnotation(
  name="checklist_class_global", 
  value=lb_types.Checklist(
      answer = [
        lb_types.ClassificationAnswer(
            name = "first_checklist_answer"
        ), 
        lb_types.ClassificationAnswer(
            name = "second_checklist_answer"
        )
      ]
    )
 )]

nested_radio_annotation =[lb_types.ClassificationAnnotation(
  name="nested_radio_question",
  value=lb_types.Radio(
    answer=lb_types.ClassificationAnswer(
      name="first_radio_answer",
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_radio_question",
          value=lb_types.Radio(
            answer=lb_types.ClassificationAnswer(
              name="first_sub_radio_answer"
            )
          )
        )
      ]
    )
  )
)]

nested_checklist_annotation = [lb_types.ClassificationAnnotation(
  name="nested_checklist_question",
  value=lb_types.Checklist(
    answer=[lb_types.ClassificationAnswer(
      name="first_checklist_answer",
      classifications=[
        lb_types.ClassificationAnnotation(
          name="sub_checklist_question",
          value=lb_types.Checklist(
            answer=[lb_types.ClassificationAnswer(
            name="first_sub_checklist_answer"
          )]
        ))
      ]
    )]
  )
)]

bbox_dm2 = {
  "top": 146.0,
  "left": 98.0,
  "height": 382.0,
  "width": 341.0
}
frame_bbox_with_checklist_subclass = [
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=10,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]), # x= left + width , y = top + height
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=11,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]), # x= left + width , y = top + height
      ),
    classifications=[
                lb_types.ClassificationAnnotation(
                    name="checklist_class",
                    value=lb_types.Checklist(answer=[lb_types.ClassificationAnswer(
                        name="first_checklist_answer"
                    )])
                )
            ]
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_class",  
    keyframe=True,
    frame=13,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm2["left"], y=bbox_dm2["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm2["left"] + bbox_dm2["width"], y=bbox_dm2["top"] + bbox_dm2["height"]), # x= left + width , y = top + height
      ),
    classifications=[
              lb_types.ClassificationAnnotation(
                  name="checklist_class",
                  value=lb_types.Checklist(answer=[lb_types.ClassificationAnswer(
                      name="second_checklist_answer"
                  )])
              )
          ]
  )
]


bbox_dm = {
  "top":617,
  "left":1371,
  "height":419,
  "width":505
}
bbox_annotation = [
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=13,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]), # x = left, y = top 
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]), # x= left + width , y = top + height
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=15,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]),
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]),
      )
  ),
  lb_types.VideoObjectAnnotation(
    name = "bbox_video",  
    keyframe=True,
    frame=19,
    segment_index=0,
    value = lb_types.Rectangle(
          start=lb_types.Point(x=bbox_dm["left"], y=bbox_dm["top"]), 
          end=lb_types.Point(x=bbox_dm["left"] + bbox_dm["width"], y=bbox_dm["top"] + bbox_dm["height"]),
      )
  )
]


text_annotation = [lb_types.ClassificationAnnotation(
  name="free_text",  # must match your ontology feature's name
  value=lb_types.Text(answer="sample text")
)]



##### 7.4. Create the label object

In [ ]:
# Create a Label object by identifying the applicable data row in Labelbox and providing a list of annotations

labels = []
annotations_list = [
          checklist_annotation, 
          radio_annotation,
          bbox_annotation, 
          frame_bbox_with_checklist_subclass,
          point_annotation, 
          polyline_annotation,
          global_checklist_annotation,
          global_radio_annotation,
          nested_checklist_annotation,
          nested_radio_annotation,
          text_annotation
      ]

flatten_list_annotations = [ann for ann_sublist in annotations_list for ann in ann_sublist] 

labels.append(
    lb_types.Label(
        data=lb_types.VideoData(global_key=global_key),
        annotations = flatten_list_annotations
    )
)

##### 7.5. Upload annotations to the project using Label Import

In [ ]:
upload_job_annotation = lb.LabelImport.create_from_objects(
    client = client,
    project_id = project.uid,
    name="video_annotations_import_" + str(uuid.uuid4()),
    labels=labels)

upload_job_annotation.wait_until_done()
# Errors will appear for annotation uploads that failed.
print("Errors:", upload_job_annotation.errors)
print("Status of uploads: ", upload_job_annotation.statuses)

##### 7.6. Send the annotations to the Model Run

In [ ]:
# get the labels id from the project
model_run.upsert_labels(project_id=project.uid)

## Optional deletions for cleanup 


In [ ]:
# project.delete()
# dataset.delete()